In [1]:
import os, time, requests, tempfile, numpy as np, nibabel as nib, gc
from typing import List, Optional

class OpenNeuroStreamer:
    # ... (sınıfın tam kodu) ...
    def __init__(self, dataset_id: str = "ds002306"): self.dataset_id = dataset_id; self.base_url = f"https://s3.amazonaws.com/openneuro.org/{dataset_id}"
    def check_url_exists(self, url: str) -> bool:
        try: response = requests.head(url, timeout=15); return response.status_code == 200
        except: return False
    def stream_nii_file(self, url: str) -> Optional[np.ndarray]:
        print(f"Streaming: {os.path.basename(url)}"); temp_path = None
        try:
            response = requests.get(url, stream=True, timeout=120); response.raise_for_status()
            with tempfile.NamedTemporaryFile(suffix='.nii.gz', delete=False) as temp_file:
                temp_path = temp_file.name
                for chunk in response.iter_content(chunk_size=8192): temp_file.write(chunk)
            img = nib.load(temp_path); return img.get_fdata(dtype=np.float32)
        except Exception as e: print(f"Error streaming file: {e}"); return None
        finally:
            if temp_path and os.path.exists(temp_path): os.unlink(temp_path)
    def load_files_for_chunk(self, subjects, num_files, urls):
        data = [];
        for s in subjects:
            if len(data) >= num_files: break
            for r in range(1, 6):
                if len(data) >= num_files: break
                url = f"{self.base_url}/{s}/func/{s}_task-training_run-{r:02d}_bold.nii.gz"
                if url in urls: continue
                if self.check_url_exists(url):
                    d = self.stream_nii_file(url)
                    if d is not None: data.append(d); urls.add(url)
        return data, urls

# --- MAIN ---
print("NOTEBOOK 1A: Downloading Chunk 1 (Files 1-20)")
streamer = OpenNeuroStreamer()
subjects_pool = [f"sub-{i:02d}" for i in range(1, 11)]
chunk_data, _ = streamer.load_files_for_chunk(subjects_pool, 20, set())
if chunk_data:
    np.save("/kaggle/working/fmri_dataset_chunk_1.npy", np.array(chunk_data, dtype=np.float32))
    print("\nChunk 1 saved successfully.")

NOTEBOOK 1A: Downloading Chunk 1 (Files 1-20)
Streaming: sub-01_task-training_run-01_bold.nii.gz
Streaming: sub-01_task-training_run-02_bold.nii.gz
Streaming: sub-01_task-training_run-03_bold.nii.gz
Streaming: sub-01_task-training_run-04_bold.nii.gz
Streaming: sub-01_task-training_run-05_bold.nii.gz
Streaming: sub-02_task-training_run-01_bold.nii.gz
Streaming: sub-02_task-training_run-02_bold.nii.gz
Streaming: sub-02_task-training_run-03_bold.nii.gz
Streaming: sub-02_task-training_run-04_bold.nii.gz
Streaming: sub-02_task-training_run-05_bold.nii.gz
Streaming: sub-03_task-training_run-01_bold.nii.gz
Streaming: sub-03_task-training_run-02_bold.nii.gz
Streaming: sub-03_task-training_run-03_bold.nii.gz
Streaming: sub-03_task-training_run-04_bold.nii.gz
Streaming: sub-03_task-training_run-05_bold.nii.gz
Streaming: sub-04_task-training_run-01_bold.nii.gz
Streaming: sub-04_task-training_run-02_bold.nii.gz
Streaming: sub-04_task-training_run-03_bold.nii.gz
Streaming: sub-04_task-training_run-